In [7]:
search_for = 79
start_from = 0
threads = 1
thread = 0
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.00445866584777832
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 5
all cases: 39449441
type: [1, 1, 1, 1, 79] 79
cases of this type: 38950081
10000 7.023898405961335
20000 10.584341539844333
30000 23.789364346323154
40000 19.726902494260347
50000 19.999976211585043
60000 20.76256200250759
70000 20.25597331274118
80000 20.28265230375091
90000 19.321389274021804
100000 19.68488341976053
110000 19.659922306378842
120000 20.825826205127694
130000 22.181256523579904
140000 20.55170398704684
150000 19.249936543287674
160000 19.938902884365035
170000 18.53573822724044
180000 18.90854414658209
190000 19.504448529102447
200000 19.334844834556996
210000 18.906774454506948
220000 19.32183641680979
230000 22.257974398069223
240000 5.142310940794938
250000 3.4968362439962033
260000 3.4895270197964505
270000 18.941553045192016
280000 19.550895387814037
290000 18.98101095041293
300000 18.870556372673185
310000 19.03387972

3040000 15.192736168998875
3050000 15.115589983343325
3060000 15.387501647763994
3070000 14.85211979869584
3080000 13.872504392871141
3090000 14.708502800785567
3100000 15.686551652556934
3110000 15.948424024001401
3120000 15.123294697428856
3130000 14.88326576984759
3140000 14.855533741686143
3150000 14.857585116442323
3160000 14.392710314802581
3170000 13.857821039058745
3180000 13.448120889523281
3190000 15.189777662419
3200000 2.9871161335284113
3210000 3.0691958259243766
3220000 2.9333059794975718
3230000 15.440872147652644
3240000 12.793820502953452
3250000 14.857316223923167
3260000 14.217766621296343
3270000 13.607254181900377
3280000 13.252127828116045
3290000 15.644910783469479
3300000 14.896440454015654
3310000 15.616901419973493
3320000 15.627409249467275
3330000 14.928112493429126
3340000 15.281226952351558
3350000 14.232108822729826
3360000 15.319378607584696
3370000 15.044337746777463
3380000 14.518613616441112
3390000 15.012600316076364
3400000 12.330641438112313
341000

6110000 12.38768982663099
6120000 12.080886863764233
6130000 12.305913351084676
6140000 12.797439034516275
6150000 11.41610896372342
6160000 2.59868789404434
6170000 2.8245526905001204
6180000 4.386637408863724
6190000 14.3376797289274
6200000 12.35111681304568
6210000 12.82183046764407
6220000 11.929407854118349
6230000 12.170716021670998
6240000 12.328314816369843
6250000 13.741882667758272
6260000 14.337587317107737
6270000 12.969126784867328
6280000 12.860564251356749
6290000 13.234306420387842
6300000 14.144261289481493
6310000 13.133328956790926
6320000 11.615570505100608
6330000 6.080861253542324
6340000 6.170039755743497
6350000 9.835858919801911
6360000 14.417430787936752
6370000 12.703391180627756
6380000 13.447925375660637
6390000 13.60413283744235
6400000 14.046458016423164
6410000 13.869037559284328
6420000 14.23340539935576
6430000 13.191380802008059
6440000 13.923083287082118
6450000 13.155458939644994
6460000 13.21091193798525
6470000 13.430960793708385
6480000 13.74691

9190000 11.415415475074509
9200000 10.668487072401225
9210000 10.324859071195629
9220000 11.186124354256783
9230000 10.94371539153592
9240000 9.378887612349013
9250000 4.990359724584056
9260000 5.519335638934016
9270000 6.659359006058123
9280000 13.448403900314146
9290000 11.51409946635133
9300000 11.98821588304882
9310000 12.907415859005903
9320000 11.922311324283005
9330000 11.211396424946209
9340000 11.777035388343505
9350000 12.315572938851258
9360000 12.440897938234013
9370000 13.919732980990398
9380000 14.750761477138996
9390000 11.521798459902577
9400000 12.85310801782245
9410000 12.257643059112588
9420000 12.515848462677583
9430000 12.004412201442292
9440000 11.809099313483
9450000 12.098271319641325
9460000 10.665267658147215
9470000 13.692190076836408
9480000 8.306073696927932
9490000 4.0471339825446675
9500000 4.656896133037765
9510000 8.284808821396517
9520000 11.361396824823181
9530000 10.189259980845907
9540000 9.644182442684505
9550000 11.443302577316874
9560000 11.04780

12200000 10.235845973624906
12210000 10.620539583629714
12220000 9.21050398197788
12230000 10.39295823572598
12240000 9.888299616115093
12250000 10.012717163671558
12260000 11.367318062992396
12270000 10.454406700787061
12280000 11.790162556977833
12290000 9.858217739052634
12300000 10.750689601736584
12310000 10.389797156712094
12320000 11.056058480409481
12330000 10.98624938215894
12340000 11.353393934957584
12350000 10.738305050013004
12360000 9.966662310062958
12370000 10.918480392267963
12380000 10.46666915225784
12390000 10.369602953911059
12400000 10.707214783776852
12410000 10.977992942308289
12420000 10.405580362670944
12430000 9.216068565944743
12440000 10.27624117196095
12450000 8.392591820275962
12460000 10.06430202303407
12470000 10.494583511017481
12480000 4.075533701016783
12490000 4.189713475196004
12500000 3.832815190806846
12510000 10.654567764462671
12520000 9.906856794340365
12530000 10.192987939685107
12540000 11.148990710963025
12550000 9.450181078464793
12560000 

...

27300000 4.946226300693545
27310000 4.472788684529146
27320000 5.113653807321608
27330000 4.917888622231344
27340000 4.782829820282877
27350000 4.296569785549323
27360000 0.997993632744425
27370000 1.0356694717340602
27380000 1.5247288625332118
27390000 5.025058676004748
27400000 4.518312864360366
27410000 4.3267986539878445
27420000 4.941481968721661
27430000 4.7196646907093065
27440000 4.331552501786927
27450000 4.621627162983132
27460000 1.9100428061190382
27470000 1.8165831950077416
27480000 2.0437792217452726
27490000 4.585120217712104
27500000 4.161777023714145
27510000 4.29989162067406
27520000 4.7515416840729445
27530000 4.658240915892303
27540000 4.569885107495183
27550000 4.901032332985726
27560000 4.957360385884166
27570000 5.223142156918075
27580000 5.142800331377473
27590000 4.965040875567376
27600000 4.491854125495725
27610000 5.114682122747448
27620000 4.690156949137669
27630000 4.492048886560116
27640000 5.179177958551514
27650000 5.244619856364251
27660000 5.0718263481

30300000 3.2187542384488452
30310000 2.8017568879776
30320000 0.7498277890230418
30330000 0.690294619720896
30340000 1.5155631682534414
30350000 3.325772885606051
30360000 3.0186843871225713
30370000 3.6657378573617074
30380000 3.0989891660992703
30390000 3.592543585329957
30400000 2.722406144638439
30410000 3.1675843756930826
30420000 2.8084340049335297
30430000 2.964181740744412
30440000 3.869304101653397
30450000 2.1676433720355566
30460000 1.4160903562993474
30470000 1.326311413163344
30480000 2.326568953908927
30490000 3.804471280719075
30500000 3.487976622152408
30510000 3.1780595129868257
30520000 3.5050603845939636
30530000 3.203821332578361
30540000 3.480312522780716
30550000 3.592465754286786
30560000 3.656558331022839
30570000 3.8103088875140347
30580000 3.5805107495756476
30590000 3.355922760569215
30600000 3.1860820781864985
30610000 3.2385095916578437
30620000 3.3446820809620026
30630000 3.3016303822308184
30640000 3.485997708378143
30650000 3.3913594334278705
30660000 2.

33280000 0.45614459961031545
33290000 0.5229578856849273
33300000 1.4754473561204209
33310000 2.3372536292576727
33320000 2.1951193192695975
33330000 2.24281095374406
33340000 2.1349150180497634
33350000 1.6495071467407345
33360000 2.4167191748756234
33370000 2.470212613523649
33380000 2.35582850139449
33390000 2.085438727217224
33400000 2.2528446348718534
33410000 2.532930855488062
33420000 2.440027808606068
33430000 1.95054920322154
33440000 2.728940883887569
33450000 0.9192522567784521
33460000 0.6149242891330057
33470000 0.8161173132758737
33480000 2.4334941396108736
33490000 2.3291336161574256
33500000 2.2976056195462347
33510000 2.3027224640685318
33520000 2.4929477751528024
33530000 2.280550179097016
33540000 2.4437162181276615
33550000 1.9329838266610573
33560000 2.4126237496190868
33570000 2.1893956140310102
33580000 2.0009602910916144
33590000 0.9401553115192055
33600000 1.0860421471843653
33610000 1.4107535475438568
33620000 2.236050160054525
33630000 2.2746508095875515
3364

36240000 0.2544206518946621
36250000 0.26902173620393544
36260000 0.8930383192694188
36270000 1.124821557022068
36280000 1.0753360741458469
36290000 1.066866170656085
36300000 1.1763050006140205
36310000 1.1400053123707043
36320000 1.0799519714257517
36330000 1.0502489387329552
36340000 1.180664709842364
36350000 1.1337535846164624
36360000 1.072676021672123
36370000 1.2530460239724053
36380000 1.1047505977528096
36390000 1.1036267592492368
36400000 1.193521821081678
36410000 1.226939038308203
36420000 1.1725112307417591
36430000 1.246460101850139
36440000 0.8240016344624957
36450000 0.4240557008544603
36460000 0.4829372007322311
36470000 0.6362354480329155
36480000 1.2362041058542197
36490000 1.4045376957841513
36500000 1.1651591528938612
36510000 0.47279008992778593
36520000 0.49104680092234737
36530000 0.494900372557203
36540000 1.333162470387492
36550000 1.1473807863498264
36560000 1.1397077647613882
36570000 0.927343487870329
36580000 0.979237643745588
36590000 0.9503460553501843


39130000 0.020442350836800204
39140000 0.020783310860474902
39150000 0.020109947027650146
39160000 0.01340196574415101
39170000 0.016628750861525536
39180000 0.015371715125514402
39190000 0.01687524167941014
39200000 0.016148044357756772
39210000 0.01511384597069025
39220000 0.014888806499322256
39230000 0.014289932880043984
39240000 0.013871780741731326
39250000 0.013268266037848262
39260000 0.01077226074117422
39270000 0.01117290552530024
39280000 0.01096444650922219
39290000 0.007094488818069298
39300000 0.007640839661306805
39310000 0.00697938964429829
39320000 0.007458979633490245
39330000 0.007693403369638655
39340000 0.007182047360062599
39350000 0.00644443315154314
39360000 0.005897846394558747
39370000 0.004291641216807895
39380000 0.004164687751154105
39390000 0.002734264181666904
39400000 0.0032195731157395576
39410000 0.002570263005435467
39420000 0.0016110410915149583
39430000 0.0012985928452677196
39440000 0.0006370297338962555
best so far: 0
type: [1, 1, 79, 1, 1] 79
cas